# Praktikum SW04

## Aufgabe 2

Gegeben ist die Entladungskurve eines RC-Netzwerk Kondensators mit der Kapazität 𝐶. Der Innenwiderstand 𝑅𝐶 beträgt 𝑅𝐶 = 100Ω.


| Zeit [ms] | Spannung [V]|
|----------|-------------|
| 0.0   |  5.0  |
| 0.03  |  2.94 |
| 0.05  |  1.73 |
| 0.08  |  1.01 |
| 0.10  |  0.6  |


Bestimmen Sie mit Hilfe eines 𝑅𝐶-Glieds und der linearen Ausgleichsrechnung die Kapazität 𝐶 des Kondensators. 

1. Wie lautet die Differentialgleichung? Berechnen Sie die allgemeine Lösung als Basis für das Modell.
2. Wie lautet die Systemmatrix?
3. Wie gross ist die Kapazität?

### 1. ODE
Die ODE lautet wie folgt:

$\dot{U}(t)=U_0-\frac{-1}{RC}\cdot U(t)$

Mit der allgemeinen Lösung:

$U(t)=U_0 \cdot e^{\frac{-1}{RC} \cdot t}$

### 2. Systemmatrix



In [147]:
# ------------ Lösung mit festem Wert für U0 ----------------

import numpy as np
import sympy as sy

# Input der Daten
t = np.array([0,0.00003,0.00005,0.00008,0.0001])  # Zeitvektor
U = np.array([5,2.94,1.73,1.01,0.6])  # Spannungsvektor
R = 100                               # Innenwiderstand über den der Kondensator entladen wird
x = sy.S('x')        # Parametervektor (tau)


A  = -t                # Systemmatrix
At = A.transpose()    # transponierte der Systemmatrix
AtA= np.dot(A,At)    # Matrixmultiplikation Systemmatrix mit ihrer Transponierten
b  = np.log(U/U[0])   # b-Vektor
Atb= np.dot(At,b)    # Matrixmultiplikation Systemmatrix mit b-Vektor


ngl = AtA*x - Atb        # Normalengleichung
tau = sy.solve(ngl , x)  # ... und deren Lösung

tau = tau[0]             # Das tau aus dem Vektor rausholen

C = 1/(R*tau)

print(tau)
print(C)

20655.6583472328
4.84128844111118e-7


In [146]:
# ------------ Lösung mit regressiertem Wert für U0 ----------

import numpy as np
import sympy as sy
import scipy as sc
import math as ma
from numpy.linalg import cholesky
from scipy.linalg import solve_triangular

# Input der Daten
t = np.array([0,0.00003,0.00005,0.00008,0.0001])  # Zeitvektor
U = np.array([5,2.94,1.73,1.01,0.6])  # Spannungsvektor
R = 100                               # Innenwiderstand über den der Kondensator entladen wird


At = np.array([[1, 1, 1, 1, 1],-t])
A  = np.transpose(At)   # transponierte der Systemmatrix


AtA= np.dot(At,A)    # Matrixmultiplikation Systemmatrix mit ihrer Transponierten
b  = np.log(U)       # b-Vektor
Atb= np.dot(At,b)    # Matrixmultiplikation Systemmatrix mit b-Vektor
x  = np.transpose(np.array([np.log(U[0]),tau])) # Parametervektor


L = cholesky(AtA)
Lt= np.transpose(L)
y = solve_triangular(L,Atb,lower=True)
y = solve_triangular(Lt,y)


tau = y[1]

C = 1/(R*tau)

U0 = ma.exp(y[0])

print(tau)
print(C)
print(U0)


21139.020020730026
4.730588262934364e-07
5.1874786140433855
